In [62]:
import os, re, requests, json
from google.cloud import secretmanager
import pandas as pd
pd.options.display.width = 140

For each neighbourhood in AB_NYC_2019.csv, get coordinates of a listing

In [127]:
bnb_df = pd.read_csv("../MP_csv_files/AB_NYC_2019.csv")
bnb_df = bnb_df[['id','neighbourhood_group','neighbourhood','latitude','longitude']]

In [128]:
# Navy Yard coordinates of first_listing fail to fit any NTA region;
# Little Neck and Woodlawn coordinates of last_listing fail to fit any NTA region;
# Thus, first and last are gathered here, and one will be dropped after NTA lookup.

bnb_sample = []
for nhood in bnb_df.neighbourhood.unique():
    
    first_listing = bnb_df.loc[bnb_df.neighbourhood == nhood][:1]
    last_listing = bnb_df.loc[bnb_df.neighbourhood == nhood][-1:]

    for listing in [first_listing, last_listing]:
        exemplar = {
            'bnb_ref_id': listing.values[0][0],
            'bnb_boro': listing.values[0][1],
            'bnb_nhood': listing.values[0][2],
            'latitude': listing.values[0][3],
            'longitude': listing.values[0][4]
        }

        bnb_sample.append(exemplar)

In [130]:
len(bnb_sample)

442

___


Align each neighbourhood with NTA2020 code, using representative listing's coordinates

In [63]:
def access_secret_version(project_id, secret_id, version_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

In [71]:
PROJECT_ID = os.getenv("PROJECT_ID")
NYC_APP_TOKEN = access_secret_version(PROJECT_ID, "NYC_APP_TOKEN", "latest")
MAP_URL = access_secret_version(PROJECT_ID, "MAP_URL", "latest") 

In [105]:
def lookup_nta(latitude, longitude):
    nta = {}

    url = f"{MAP_URL}?$where=intersects(the_geom, 'POINT({longitude} {latitude})')"
    headers = {"X-App-Token": NYC_APP_TOKEN}
    response = requests.get(url, headers=headers)
    jsn = json.loads(response.text)

    fields = [
        'boroname',
        'countyfips',
        'nta2020',
        'ntaname',
        'cdta2020',
        'cdtaname']
        
    if jsn:
        for field in fields:
            nta[field] = jsn[0][field]
    return nta

In [131]:
# Could do this for every listing individually, 
# but ~50,000 API calls might get throttled.
# For now, this will do.

nta_lookup = []
for nhood in bnb_sample:
    nta = lookup_nta(nhood['latitude'], nhood['longitude'])
    nhood.update(nta)
    nta_lookup.append(nhood)
nta_df = pd.DataFrame(nta_lookup)

___

Quick check on bnb listings' fit to NTA map

In [132]:
nta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bnb_ref_id  442 non-null    int64  
 1   bnb_boro    442 non-null    object 
 2   bnb_nhood   442 non-null    object 
 3   latitude    442 non-null    float64
 4   longitude   442 non-null    float64
 5   boroname    439 non-null    object 
 6   countyfips  439 non-null    object 
 7   nta2020     439 non-null    object 
 8   ntaname     439 non-null    object 
 9   cdta2020    439 non-null    object 
 10  cdtaname    439 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 38.1+ KB


In [133]:
nta_df.loc[nta_df.boroname.isna()==True]

,bnb_ref_id,bnb_boro,bnb_nhood,latitude,longitude,boroname,countyfips,nta2020,ntaname,cdta2020,cdtaname
131,33853642,Bronx,Woodlawn,40.90406,-73.86286,NaN,NaN,NaN,NaN,NaN,NaN
292,3079125,Brooklyn,Navy Yard,40.70481,-73.97755,NaN,NaN,NaN,NaN,NaN,NaN
421,35515780,Queens,Little Neck,40.77444,-73.73373,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
nta_df = nta_df.dropna()

In [139]:
nta_df.drop_duplicates(subset='bnb_nhood', keep='first').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 0 to 440
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bnb_ref_id  221 non-null    int64  
 1   bnb_boro    221 non-null    object 
 2   bnb_nhood   221 non-null    object 
 3   latitude    221 non-null    float64
 4   longitude   221 non-null    float64
 5   boroname    221 non-null    object 
 6   countyfips  221 non-null    object 
 7   nta2020     221 non-null    object 
 8   ntaname     221 non-null    object 
 9   cdta2020    221 non-null    object 
 10  cdtaname    221 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 20.7+ KB


In [140]:
nta_df = nta_df.drop_duplicates(subset='bnb_nhood', keep='first')

In [141]:
# nta_df.to_csv("bnb_nhood_sample_coords_nta.csv")

___
Link bnb neighbourhoods with demographics of interest

In [227]:
df_profiles = pd.read_csv("demographics_by_nta_2019.csv", index_col=0)

In [228]:
df_profiles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262 entries, 0 to 261
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GeoID      262 non-null    object 
 1   MdGRE      211 non-null    object 
 2   MdVlE      209 non-null    object 
 3   EA_BchDHP  216 non-null    float64
 4   Pop_1E     262 non-null    object 
dtypes: float64(1), object(4)
memory usage: 12.3+ KB


In [230]:
side_input = pd.merge(nta_df, df_profiles, how="left", left_on="nta2020", right_on="GeoID")


In [231]:
side_input

,bnb_ref_id,bnb_boro,bnb_nhood,latitude,longitude,boroname,countyfips,nta2020,ntaname,cdta2020,cdtaname,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
0,2539,Brooklyn,Kensington,40.64749,-73.97237,Brooklyn,047,BK1203,Kensington,BK12,BK12 Borough Park-Kensington (CD 12 Approximat...,BK1203,"1,607","693,439",46.7,"38,788"
1,2595,Manhattan,Midtown,40.75362,-73.98377,Manhattan,061,MN0502,Midtown-Times Square,MN05,MN05 Midtown-Flatiron-Union Square (CD 5 Appro...,MN0502,"2,931","1,098,496",78.2,"19,860"
2,3647,Manhattan,Harlem,40.80902,-73.94190,Manhattan,061,MN1002,Harlem (North),MN10,MN10 Harlem (CD 10 Equivalent),MN1002,"1,089","609,513",35.1,"82,712"
3,21741106,Brooklyn,Clinton Hill,40.69479,-73.96812,Brooklyn,047,BK0204,Clinton Hill,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,BK0204,"1,971","730,698",65.8,"27,082"
4,5022,Manhattan,East Harlem,40.79851,-73.94399,Manhattan,061,MN1102,East Harlem (North),MN11,MN11 East Harlem (CD 11 Equivalent),MN1102,873,"612,027",27.5,"64,762"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,22856025,Staten Island,Bull's Head,40.60295,-74.17117,Staten Island,085,SI0204,New Springville-Willowbrook-Bulls Head-Travis,SI02,SI02 Mid-Island (CD 2 Approximation),SI0204,"1,551","498,178",34.0,"43,057"
217,23597323,Staten Island,New Dorp,40.57044,-74.11747,Staten Island,085,SI0202,New Dorp-Midland Beach,SI02,SI02 Mid-Island (CD 2 Approximation),SI0202,"1,536","484,631",29.7,"27,538"
218,26258351,Staten Island,Rossville,40.54790,-74.21017,Staten Island,085,SI0303,Arden Heights-Rossville,SI03,SI03 South Shore (CD 3 Approximation),SI0303,"1,242","444,250",32.8,"30,900"
219,26703677,Queens,Breezy Point,40.56605,-73.86994,Queens,081,QN8492,Jacob Riis Park-Fort Tilden-Breezy Point Tip,QN84,QN84 Jamaica Bay (East) (JIA 84 Approximation),QN8492,NaN,"749,999",27.3,11


In [232]:
side_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 0 to 220
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bnb_ref_id  221 non-null    int64  
 1   bnb_boro    221 non-null    object 
 2   bnb_nhood   221 non-null    object 
 3   latitude    221 non-null    float64
 4   longitude   221 non-null    float64
 5   boroname    221 non-null    object 
 6   countyfips  221 non-null    object 
 7   nta2020     221 non-null    object 
 8   ntaname     221 non-null    object 
 9   cdta2020    221 non-null    object 
 10  cdtaname    221 non-null    object 
 11  GeoID       221 non-null    object 
 12  MdGRE       218 non-null    object 
 13  MdVlE       218 non-null    object 
 14  EA_BchDHP   219 non-null    float64
 15  Pop_1E      221 non-null    object 
dtypes: float64(3), int64(1), object(12)
memory usage: 29.4+ KB


In [233]:
side_input[side_input.MdGRE.isna()]

,bnb_ref_id,bnb_boro,bnb_nhood,latitude,longitude,boroname,countyfips,nta2020,ntaname,cdta2020,cdtaname,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
37,32363,Queens,Flushing,40.74028,-73.83168,Queens,081,QN0871,Mount Hebron & Cedar Grove Cemeteries,QN08,QN08 Fresh Meadows-Hillcrest-Briarwood (CD 8 A...,QN0871,NaN,NaN,NaN,0
161,5288228,Bronx,Tremont,40.83875,-73.88728,Bronx,005,BX0391,Crotona Park,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,BX0391,NaN,NaN,NaN,0
219,26703677,Queens,Breezy Point,40.56605,-73.86994,Queens,081,QN8492,Jacob Riis Park-Fort Tilden-Breezy Point Tip,QN84,QN84 Jamaica Bay (East) (JIA 84 Approximation),QN8492,NaN,"749,999",27.3,11


In [234]:
side_input[side_input.MdVlE.isna()]


,bnb_ref_id,bnb_boro,bnb_nhood,latitude,longitude,boroname,countyfips,nta2020,ntaname,cdta2020,cdtaname,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
37,32363,Queens,Flushing,40.74028,-73.83168,Queens,081,QN0871,Mount Hebron & Cedar Grove Cemeteries,QN08,QN08 Fresh Meadows-Hillcrest-Briarwood (CD 8 A...,QN0871,NaN,NaN,NaN,0
132,1839818,Manhattan,Stuyvesant Town,40.73196,-73.97932,Manhattan,061,MN0601,Stuyvesant Town-Peter Cooper Village,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,MN0601,"2,524",NaN,81.5,"22,335"
161,5288228,Bronx,Tremont,40.83875,-73.88728,Bronx,005,BX0391,Crotona Park,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,BX0391,NaN,NaN,NaN,0


In [235]:
side_input[side_input.EA_BchDHP.isna()]


,bnb_ref_id,bnb_boro,bnb_nhood,latitude,longitude,boroname,countyfips,nta2020,ntaname,cdta2020,cdtaname,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
37,32363,Queens,Flushing,40.74028,-73.83168,Queens,081,QN0871,Mount Hebron & Cedar Grove Cemeteries,QN08,QN08 Fresh Meadows-Hillcrest-Briarwood (CD 8 A...,QN0871,NaN,NaN,NaN,0
161,5288228,Bronx,Tremont,40.83875,-73.88728,Bronx,005,BX0391,Crotona Park,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,BX0391,NaN,NaN,NaN,0


Check on Breezy Point, Tremont, Stuyvesant, Flushing.

___
Breezy Point?

In [156]:
bnb_df.loc[bnb_df.neighbourhood == "Breezy Point"]


,id,neighbourhood_group,neighbourhood,latitude,longitude
33698,26703677,Queens,Breezy Point,40.56605,-73.86994
34429,27301638,Queens,Breezy Point,40.56568,-73.87009
48751,36417725,Queens,Breezy Point,40.56546,-73.86968


In [157]:
lookup_nta(40.56568, -73.87009)

{'boroname': 'Queens',
 'countyfips': '081',
 'nta2020': 'QN8492',
 'ntaname': 'Jacob Riis Park-Fort Tilden-Breezy Point Tip',
 'cdta2020': 'QN84',
 'cdtaname': 'QN84 Jamaica Bay (East) (JIA 84 Approximation)'}

In [158]:
lookup_nta(40.56546,-73.86968)

{'boroname': 'Queens',
 'countyfips': '081',
 'nta2020': 'QN8492',
 'ntaname': 'Jacob Riis Park-Fort Tilden-Breezy Point Tip',
 'cdta2020': 'QN84',
 'cdtaname': 'QN84 Jamaica Bay (East) (JIA 84 Approximation)'}

In [159]:
lookup_nta(40.56605,-73.86994)

{'boroname': 'Queens',
 'countyfips': '081',
 'nta2020': 'QN8492',
 'ntaname': 'Jacob Riis Park-Fort Tilden-Breezy Point Tip',
 'cdta2020': 'QN84',
 'cdtaname': 'QN84 Jamaica Bay (East) (JIA 84 Approximation)'}

All three bnb Breezy Point listings are located within a single NTA,<br>
for which rent is NaN and population is eleven, according to my NTA sources;<br>
PUMA demographics might exist, but then granularity of demographics would be mixed,<br>
some NTA and some PUMA.<br>
I prefer finer granularity, at expense of Breezy Point stats.<br><br>

According to map, QN1403 borders the missing data to the east and west,<br>
and NTA demographics exist for QN1403. Breezy Point solved.

In [250]:
df_profiles[df_profiles.GeoID=='QN8492']

,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
255,QN8492,NaN,"749,999",27.3,11


In [229]:
df_profiles[df_profiles.GeoID=='QN1403']

,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
180,QN1403,"1,418","615,257",45.0,"24,768"


In [252]:
(27.3/100)*11

3.003

In [251]:
24768+11

24779

In [254]:
(24768*(45/100) + 3) / 24779

0.4499213043302797

In [236]:
side_input.loc[side_input.bnb_nhood == "Breezy Point", "GeoID"] = 'QN1403'
side_input.loc[side_input.bnb_nhood == "Breezy Point", "MdGRE"] = '1,418'
side_input.loc[side_input.bnb_nhood == "Breezy Point", "MdVlE"] = '615,257'
side_input.loc[side_input.bnb_nhood == "Breezy Point", "EA_BchDHP"] = 45.0
side_input.loc[side_input.bnb_nhood == "Breezy Point", "Pop_1E"] = '24,768'

___
Tremont? <br>
At least one listing is too close to a park, with 0 people and NaN for other stats.

In [237]:
bnb_df.loc[bnb_df.neighbourhood == "Tremont"]

,id,neighbourhood_group,neighbourhood,latitude,longitude
7226,5288228,Bronx,Tremont,40.83875,-73.88728
7402,5495927,Bronx,Tremont,40.83887,-73.88805
7636,5749147,Bronx,Tremont,40.84061,-73.88858
16538,13248565,Bronx,Tremont,40.84268,-73.89294
16677,13335424,Bronx,Tremont,40.84559,-73.89815
22216,17899730,Bronx,Tremont,40.84546,-73.89013
22708,18375815,Bronx,Tremont,40.84647,-73.89299
23082,18690466,Bronx,Tremont,40.84594,-73.89358
39256,30592546,Bronx,Tremont,40.84289,-73.88773
41503,32263166,Bronx,Tremont,40.84740,-73.89612


In [238]:
tm = bnb_df.loc[bnb_df.neighbourhood == "Tremont"]

In [239]:
tm_lookups = []
for lat, lon in zip(tm.latitude, tm.longitude):
    nta = lookup_nta(lat, lon)
    tm_lookups.append(nta)

In [240]:
tm_nta = [tm['nta2020'] for tm in tm_lookups]

In [269]:
tm_nta

['BX0391',
 'BX0391',
 'BX0602',
 'BX0602',
 'BX0602',
 'BX0602',
 'BX0602',
 'BX0602',
 'BX0602',
 'BX0602',
 'BX0602']

In [241]:
set(tm_nta) # {'BX0391', 'BX0602'}

{'BX0391', 'BX0602'}

Good, Tremont has another possible NTA for stats lookup

In [242]:
df_profiles.loc[df_profiles.GeoID == 'BX0391']


,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
233,BX0391,NaN,NaN,NaN,0


In [243]:
df_profiles.loc[df_profiles.GeoID == 'BX0602']

,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
67,BX0602,"1,048","391,332",13.7,"31,270"


In [244]:
side_input.loc[side_input.bnb_nhood == "Tremont", "GeoID"] = 'BX0602'
side_input.loc[side_input.bnb_nhood == "Tremont", "MdGRE"] = '1,048'
side_input.loc[side_input.bnb_nhood == "Tremont", "MdVlE"] = '391,332'
side_input.loc[side_input.bnb_nhood == "Tremont", "EA_BchDHP"] = 13.7
side_input.loc[side_input.bnb_nhood == "Tremont", "Pop_1E"] = '31,270'

In [255]:
side_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 0 to 220
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bnb_ref_id  221 non-null    int64  
 1   bnb_boro    221 non-null    object 
 2   bnb_nhood   221 non-null    object 
 3   latitude    221 non-null    float64
 4   longitude   221 non-null    float64
 5   boroname    221 non-null    object 
 6   countyfips  221 non-null    object 
 7   nta2020     221 non-null    object 
 8   ntaname     221 non-null    object 
 9   cdta2020    221 non-null    object 
 10  cdtaname    221 non-null    object 
 11  GeoID       221 non-null    object 
 12  MdGRE       220 non-null    object 
 13  MdVlE       219 non-null    object 
 14  EA_BchDHP   220 non-null    float64
 15  Pop_1E      221 non-null    object 
dtypes: float64(3), int64(1), object(12)
memory usage: 29.4+ KB


___
Stuyvesant?  NaN for median value of owner-occupied homes.

In [245]:
side_input.loc[side_input.bnb_nhood == "Stuyvesant Town"]


,bnb_ref_id,bnb_boro,bnb_nhood,latitude,longitude,boroname,countyfips,nta2020,ntaname,cdta2020,cdtaname,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
132,1839818,Manhattan,Stuyvesant Town,40.73196,-73.97932,Manhattan,061,MN0601,Stuyvesant Town-Peter Cooper Village,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,MN0601,"2,524",NaN,81.5,"22,335"


In [246]:
bnb_df[bnb_df.neighbourhood == "Stuyvesant Town"]

,id,neighbourhood_group,neighbourhood,latitude,longitude
3101,1839818,Manhattan,Stuyvesant Town,40.73196,-73.97932
3731,2250170,Manhattan,Stuyvesant Town,40.73205,-73.98094
6744,4837935,Manhattan,Stuyvesant Town,40.72912,-73.97624
6784,4854552,Manhattan,Stuyvesant Town,40.73001,-73.97579
9007,6914268,Manhattan,Stuyvesant Town,40.73390,-73.97743
9478,7269771,Manhattan,Stuyvesant Town,40.73050,-73.98043
12309,9511685,Manhattan,Stuyvesant Town,40.73288,-73.97984
18297,14353077,Manhattan,Stuyvesant Town,40.72940,-73.97751
18381,14443766,Manhattan,Stuyvesant Town,40.73304,-73.97421
19722,15789384,Manhattan,Stuyvesant Town,40.73167,-73.98146


In [247]:
sv = bnb_df[bnb_df.neighbourhood == "Stuyvesant Town"]

sv_lookups = []
for lat, lon in zip(sv.latitude, sv.longitude):
    nta = lookup_nta(lat, lon)
    sv_lookups.append(nta)

In [248]:
sv_nta = [sv['nta2020'] for sv in sv_lookups]


In [249]:
set(sv_nta)

{'MN0601'}

Nothing we can do about that, apparently. Unless the map helps... <br>
No, this area has no homes. Planned development with apartments, only.<br>
So Stuyvesant gets NaN for median home value.

___
Flushing?<br>
At least one listing is too close to a cemetary, with 0 people and NaN for other stats.

In [155]:
bnb_df.loc[bnb_df.neighbourhood == "Flushing"]

,id,neighbourhood_group,neighbourhood,latitude,longitude
143,32363,Queens,Flushing,40.74028,-73.83168
1271,546383,Queens,Flushing,40.75578,-73.81948
1649,748656,Queens,Flushing,40.76372,-73.79201
1710,774791,Queens,Flushing,40.75612,-73.82517
1832,818337,Queens,Flushing,40.75979,-73.81016
...,...,...,...,...,...
48419,36243764,Queens,Flushing,40.74407,-73.82645
48600,36335028,Queens,Flushing,40.74761,-73.81380
48652,36356906,Queens,Flushing,40.75825,-73.83369
48790,36427429,Queens,Flushing,40.75104,-73.81459


In [189]:
fl = bnb_df[bnb_df.neighbourhood == "Flushing"]

In [190]:
fl_lookups = []
for lat, lon in zip(fl.latitude, fl.longitude):
    nta = lookup_nta(lat, lon)
    fl_lookups.append(nta)

In [192]:
fl_nta = [fl['nta2020'] for fl in fl_lookups]

In [195]:
set(fl_nta)

# {'QN0704',
#  'QN0705',
#  'QN0706',
#  'QN0707',
#  'QN0791',
#  'QN0801',
#  'QN0802',
#  'QN0803',
#  'QN0871',
#  'QN1101',
#  'QN8191'}

{'QN0704',
 'QN0705',
 'QN0706',
 'QN0707',
 'QN0791',
 'QN0801',
 'QN0802',
 'QN0803',
 'QN0871',
 'QN1101',
 'QN8191'}

Wow, 11 different NTAs for bnb listings in Flushing.<br>
Tukey's ninthers method will do, for rent and home value -- median of medians, as informal approx of median.

In [209]:
dfw = pd.DataFrame()

for nta_code in set(fl_nta):
    
    dfw = pd.concat([dfw, df_profiles[df_profiles.GeoID == nta_code]], axis=0)

In [210]:
dfw

,GeoID,MdGRE,MdVlE,EA_BchDHP,Pop_1E
146,QN0706,"1,496","762,699",25.8,"20,164"
148,QN0801,"1,558","350,880",44.9,"34,292"
149,QN0802,"1,215","661,998",33.4,"35,756"
144,QN0704,"1,566","725,104",23.6,"54,051"
147,QN0707,"1,453","352,632",20.4,"59,788"
150,QN0803,"1,808","782,087",46.8,"22,938"
161,QN1101,"1,772","752,432",35.7,"33,671"
145,QN0705,"1,439","815,136",22.6,"30,137"


In [260]:
for col in ['MdGRE', 'MdVlE', 'Pop_1E']:
    dfw[col] = dfw[col].apply(lambda x: int(x.replace(',','')))

In [264]:
dfw_educ = sum(dfw.EA_BchDHP*dfw.Pop_1E)/sum(dfw.Pop_1E)

In [265]:
dfw_educ 
# EA_BchDHP = 29.9

29.938825022266393

In [266]:
sum(dfw.Pop_1E) 
# Pop_1E = '290,797'

290797

In [268]:
dfw.describe().loc['50%'] 
# MdGRE = '1,527'
# MdVlE = '738,768'

MdGRE          1527.0
MdVlE        738768.0
EA_BchDHP        29.6
Pop_1E        33981.5
Name: 50%, dtype: float64

In [272]:
side_input.loc[side_input.bnb_nhood == "Flushing", "GeoID"] = 'QN0706' # GeoID not used moving forward, actually
side_input.loc[side_input.bnb_nhood == "Flushing", "MdGRE"] = '1,527'
side_input.loc[side_input.bnb_nhood == "Flushing", "MdVlE"] = '738,768'
side_input.loc[side_input.bnb_nhood == "Flushing", "EA_BchDHP"] = 29.9
side_input.loc[side_input.bnb_nhood == "Flushing", "Pop_1E"] = '290,797'

___

In [271]:
side_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 0 to 220
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bnb_ref_id  221 non-null    int64  
 1   bnb_boro    221 non-null    object 
 2   bnb_nhood   221 non-null    object 
 3   latitude    221 non-null    float64
 4   longitude   221 non-null    float64
 5   boroname    221 non-null    object 
 6   countyfips  221 non-null    object 
 7   nta2020     221 non-null    object 
 8   ntaname     221 non-null    object 
 9   cdta2020    221 non-null    object 
 10  cdtaname    221 non-null    object 
 11  GeoID       221 non-null    object 
 12  MdGRE       221 non-null    object 
 13  MdVlE       220 non-null    object 
 14  EA_BchDHP   221 non-null    float64
 15  Pop_1E      221 non-null    object 
dtypes: float64(3), int64(1), object(12)
memory usage: 29.4+ KB


Only missing value is for median home value in Stuyvesant.

In [274]:
# side_input.to_csv("bnb_nta_side_input.csv")

___

Now let's format the side_input to suit our needs.

In [290]:
dicts = []
for row in range(len(side_input.index)):
    nh = {
        'nta_code': side_input['nta2020'][row],
        'median_rent': side_input['MdGRE'][row],
        'median_home_value': side_input['MdVlE'][row],
        'percent_atleast_bachdeg': side_input['EA_BchDHP'][row],
        'population': side_input['Pop_1E'][row],
        'nta_name': side_input['ntaname'][row]
    }
    elem = {side_input['bnb_nhood'][row]: {k:v for k,v in nh.items()}}
    dicts.append(elem)

In [291]:
dicts

[{'Kensington': {'nta_code': 'BK1203',
   'median_rent': '1,607',
   'median_home_value': '693,439',
   'percent_atleast_bachdeg': 46.7,
   'population': '38,788',
   'nta_name': 'Kensington'}},
 {'Midtown': {'nta_code': 'MN0502',
   'median_rent': '2,931',
   'median_home_value': '1,098,496',
   'percent_atleast_bachdeg': 78.2,
   'population': '19,860',
   'nta_name': 'Midtown-Times Square'}},
 {'Harlem': {'nta_code': 'MN1002',
   'median_rent': '1,089',
   'median_home_value': '609,513',
   'percent_atleast_bachdeg': 35.1,
   'population': '82,712',
   'nta_name': 'Harlem (North)'}},
 {'Clinton Hill': {'nta_code': 'BK0204',
   'median_rent': '1,971',
   'median_home_value': '730,698',
   'percent_atleast_bachdeg': 65.8,
   'population': '27,082',
   'nta_name': 'Clinton Hill'}},
 {'East Harlem': {'nta_code': 'MN1102',
   'median_rent': '873',
   'median_home_value': '612,027',
   'percent_atleast_bachdeg': 27.5,
   'population': '64,762',
   'nta_name': 'East Harlem (North)'}},
 {'M

In [293]:
# with open("side_input.json", "w") as f:
#     json.dump(dicts, f)